In [3]:
library(tidyverse)
library(ggpubr)
library(GGally)
library(yaml)
library(patchwork)
#library(Seurat)
#library(ggvenn)

#read config
config <- read_yaml(file = "../HiRES_preprocess_pipeline//config.yaml")

#Common
Raw <- read_table2("../stat/raw.fq.stat",col_names=F) %>% arrange(X1) %>% rowwise() %>% mutate(X2 = X2/4*300/1000000000,X1 = strsplit(X1,split = "/")[[1]][3],X1 = strsplit(X1,split = "_")[[1]][1])
DNA <- read_table2("../stat/dna.fq.stat",col_names=F) %>% arrange(X1) %>% rowwise()%>% mutate(X2 = X2/4*300/1000000000,X1 = strsplit(X1,split = "/")[[1]][3])
RNA <- read_table2("../stat/rna.fq.stat",col_names=F) %>% arrange(X1) %>% rowwise() %>% mutate(X2 = X2/4*300/1000000000,X1 = strsplit(X1,split = "/")[[1]][3])

raw_pairs <- read_table2("../stat/raw.pairs.stat",col_names=F) %>% arrange(X1) %>% rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][3])
pairs_dedup <- read_table2("../stat/pairs.dedup.stat",col_names=F) %>% arrange(X1) %>% rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][3])

pairs_c1 <- read_table2("../stat/pairs.c1.stat",col_names=F)%>% arrange(X1) %>%rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][5]) %>% mutate(X1 = str_replace(X1,".pairs.gz",""))
pairs_c12 <- read_table2("../stat/pairs.c12.stat",col_names=F)%>% arrange(X1) %>%rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][5]) %>% mutate(X1 = str_replace(X1,".pairs.gz",""))
pairs_c123 <- read_table2("../stat/pairs.c123.stat",col_names=F)%>% arrange(X1) %>%rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][5]) %>% mutate(X1 = str_replace(X1,".pairs.gz",""))
inter_pairs_c123 <- read_table2(file = "../stat/inter.pairs.c123.stat",col_names = F) %>% arrange(X1) %>%rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][5]) %>% mutate(X1 = str_replace(X1,".pairs.gz",""))

yperx <- read_table2("../stat/yperx.stat",col_names=F) %>% arrange(X1) %>% rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][2])
stat <- Raw %>% left_join(DNA,by="X1") %>% left_join(RNA,by="X1") %>% left_join(yperx,by="X1") %>% left_join(raw_pairs,by="X1") %>% left_join(pairs_dedup,by="X1") %>% 
    left_join(pairs_c1,by="X1") %>% left_join(pairs_c12,by="X1")%>% left_join(pairs_c123,by="X1") %>% left_join(inter_pairs_c123,by="X1")
RNAres <- read_table2("../result/RNA_Res/counts.gene.total.format.tsv")
featureStat <- cbind(as.data.frame(colSums(RNAres %>% select(-gene))),colSums(RNAres %>% select(-gene) != 0)) %>% rownames_to_column("X1")
RNAresExon <- read_table2("../result/RNA_Res/counts.exon.total.format.tsv")
featureStatExon <- cbind(as.data.frame(colSums(RNAresExon %>% select(-gene))),colSums(RNAresExon %>% select(-gene) != 0))  %>% rownames_to_column("X1")

if(config$if_RNA_snp_split){
    RNAresGenome1 <- read_table2("../result/RNA_Res/counts.gene.genome1.tsv")
    RNAresGenome2 <- read_table2("../result/RNA_Res/counts.gene.genome2.tsv")
    featureStatGenome1 <- cbind(as.data.frame(colSums(RNAresGenome1 %>% select(-gene))),colSums(RNAresGenome1 %>% select(-gene) != 0)) %>% rownames_to_column("X1")
    featureStatGenome2 <- cbind(as.data.frame(colSums(RNAresGenome2 %>% select(-gene))),colSums(RNAresGenome2 %>% select(-gene) != 0)) %>% rownames_to_column("X1")
    stat <- stat %>% left_join(featureStat,by="X1") %>% left_join(featureStatExon,by="X1") %>% left_join(featureStatGenome1,by="X1")  %>% left_join(featureStatGenome2,by="X1")
    names(stat) <- c("cellname","Rawreads","DNAreads","RNAreads","yperx","raw_pairs","pairs_dedup","pairs_clean1","pairs_clean2","pairs_clean3","inter_pairs_clean3",
                     "UMIs_gene","genes_gene","UMIs_exon","genes_exon","UMIs_gene_genome1","genes_gene_genome1","UMIs_gene_genome2","genes_gene_genome2")
} else{
    stat <- stat %>% left_join(featureStat,by="X1") %>% left_join(featureStatExon,by="X1") 
    names(stat) <- c("cellname","Rawreads","DNAreads","RNAreads","yperx","raw_pairs","pairs_dedup","pairs_clean1","pairs_clean2","pairs_clean3","inter_pairs_clean3",
                     "UMIs_gene","genes_gene","UMIs_exon","genes_exon")
}


if(config$if_structure){
    rmsd <- read_table2("../stat/rmsd.info",col_names = F) %>% rowwise() %>% mutate(X1=str_split(X1,pattern = fixed("/")),cellname = X1[1], res = str_split(X1[3], fixed("."))[[1]][2],TOP3_RMSD = X5) %>% select(cellname,res,TOP3_RMSD) %>% spread(res,TOP3_RMSD) %>% arrange(cellname)
    stat <- stat %>% left_join(rmsd)
}





── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)


── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)


── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)


── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)


── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)


── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)


── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)


── Column specification ──────────────────────────────

In [4]:
plot <- stat %>% mutate(RNAreadsRatio = RNAreads/(RNAreads+DNAreads),
                        pairsPerRead = raw_pairs/DNAreads/1000000000*300,
                        pairsValidRatio = pairs_clean3/raw_pairs,
                        interPairsRatio = inter_pairs_clean3/pairs_clean3) 
#plot[is.na(plot)]<-0

In [5]:
options(repr.matrix.max.cols=50, repr.matrix.max.rows=100)
plot %>% summary()

   cellname            Rawreads        DNAreads        RNAreads      
 Length:1           Min.   :1.541   Min.   :1.476   Min.   :0.05569  
 Class :character   1st Qu.:1.541   1st Qu.:1.476   1st Qu.:0.05569  
 Mode  :character   Median :1.541   Median :1.476   Median :0.05569  
                    Mean   :1.541   Mean   :1.476   Mean   :0.05569  
                    3rd Qu.:1.541   3rd Qu.:1.476   3rd Qu.:0.05569  
                    Max.   :1.541   Max.   :1.476   Max.   :0.05569  
     yperx           raw_pairs       pairs_dedup      pairs_clean1   
 Min.   :0.02448   Min.   :549679   Min.   :201060   Min.   :200110  
 1st Qu.:0.02448   1st Qu.:549679   1st Qu.:201060   1st Qu.:200110  
 Median :0.02448   Median :549679   Median :201060   Median :200110  
 Mean   :0.02448   Mean   :549679   Mean   :201060   Mean   :200110  
 3rd Qu.:0.02448   3rd Qu.:549679   3rd Qu.:201060   3rd Qu.:200110  
 Max.   :0.02448   Max.   :549679   Max.   :201060   Max.   :200110  
  pairs_clean2     p

In [6]:
plot

cellname,Rawreads,DNAreads,RNAreads,yperx,raw_pairs,pairs_dedup,pairs_clean1,pairs_clean2,pairs_clean3,inter_pairs_clean3,UMIs_gene,genes_gene,UMIs_exon,genes_exon,1m,200k,20k,50k,RNAreadsRatio,pairsPerRead,pairsValidRatio,interPairsRatio
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
OrgfE951001,1.540634,1.47555,0.0556875,0.02448,549679,201060,200110,162955,162875,22148,6087,3245,1852,1301,1.751261,3.570228,10.15922,6.744294,0.03636765,0.1117575,0.2963093,0.1359816
